In [1]:
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (
    Flatten,
    Dense,
    Dropout,
    BatchNormalization,
    LeakyReLU,
)
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping

import numpy as np
import os

# 加载MNIST数据集
(x_train, y_train), (x_test, y_test) = mnist.load_data()

# 数据预处理
x_train = x_train.astype("float32") / 255.0
x_test = x_test.astype("float32") / 255.0

x_train = np.expand_dims(x_train, axis=-1)  # 增加通道维度
x_test = np.expand_dims(x_test, axis=-1)

y_train = to_categorical(y_train, 10)
y_test = to_categorical(y_test, 10)

# 数据增强
datagen = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.3,
    height_shift_range=0.3,
    zoom_range=0.5,
    shear_range=0.5,
    fill_mode="nearest",
)
datagen.fit(x_train)

# 构建 BP 神经网络模型
# 没有池化 池化操作主要应用在CNN网络中
model = Sequential(
    [
        Flatten(input_shape=(28, 28, 1)),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(1024),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(512),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(128),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(64),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(32),
        LeakyReLU(alpha=0.1),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(10, activation="softmax"),
    ]
)

# 编译模型
model.compile(
    optimizer=tf.keras.optimizers.legacy.Adam(learning_rate=0.001), # 使用 Adam 优化器
    loss="categorical_crossentropy",  # 多分类交叉熵损失
    metrics=["accuracy"],
)

# 回调函数
reduce_lr = ReduceLROnPlateau(
    monitor="val_loss", factor=0.5, patience=10, min_lr=1e-6, verbose=1
)
early_stopping = EarlyStopping(
    monitor="val_loss", patience=15, restore_best_weights=True, verbose=1
)

# 训练模型
model.fit(
    datagen.flow(x_train, y_train, batch_size=64),
    epochs=300,
    validation_data=(x_test, y_test),
    steps_per_epoch=len(x_train) // 64,
    callbacks=[reduce_lr, early_stopping],
)

# 评估模型
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_acc}")

# 保存模型
model.save("mnist_model_bp_x4.keras")

2025-01-12 20:07:50.366667: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M2
2025-01-12 20:07:50.366698: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2025-01-12 20:07:50.366707: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2025-01-12 20:07:50.366742: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:303] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2025-01-12 20:07:50.366763: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:269] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


Epoch 1/300


2025-01-12 20:07:51.329488: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


935/937 [============================>.] - ETA: 0s - loss: 2.3989 - accuracy: 0.1493

2025-01-12 20:08:10.705736: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


937/937 [==============================] - 22s 22ms/step - loss: 2.3984 - accuracy: 0.1495 - val_loss: 2.1529 - val_accuracy: 0.2128 - lr: 0.0010
Epoch 2/300
937/937 [==============================] - 20s 22ms/step - loss: 2.1273 - accuracy: 0.2120 - val_loss: 1.8546 - val_accuracy: 0.3427 - lr: 0.0010
Epoch 3/300
937/937 [==============================] - 20s 22ms/step - loss: 2.0025 - accuracy: 0.2613 - val_loss: 1.5876 - val_accuracy: 0.4389 - lr: 0.0010
Epoch 4/300
937/937 [==============================] - 20s 22ms/step - loss: 1.8738 - accuracy: 0.3217 - val_loss: 1.3215 - val_accuracy: 0.5311 - lr: 0.0010
Epoch 5/300
937/937 [==============================] - 20s 22ms/step - loss: 1.7406 - accuracy: 0.3766 - val_loss: 1.1695 - val_accuracy: 0.6406 - lr: 0.0010
Epoch 6/300
937/937 [==============================] - 20s 22ms/step - loss: 1.6426 - accuracy: 0.4158 - val_loss: 1.0240 - val_accuracy: 0.6633 - lr: 0.0010
Epoch 7/300
937/937 [==============================] - 20s 22ms/

在 Keras 中，反向传播是由框架自动处理的。你不需要显式地编写反向传播的代码。反向传播的过程在模型编译和训练过程中自动完成。以下是代码中体现反向传播的部分：

1. **模型编译**：
在编译模型时，指定了优化器和损失函数。优化器（如 Adam）会在训练过程中自动执行反向传播，以更新模型的权重。

2. **模型训练**：
在调用 `model.fit` 方法时，Keras 会自动执行前向传播和反向传播，以最小化损失函数。



1. **前向传播**：
输入数据通过网络的各层，计算输出和损失。

2. **计算损失**：
使用指定的损失函数（如 categorical_crossentropy）计算预测值与真实值之间的差异。

3. **反向传播**：
计算损失相对于每个参数的梯度。
优化器（如 Adam）使用这些梯度更新模型的权重。

4. **权重更新**：
根据计算的梯度和学习率，优化器调整模型的权重，以最小化损失。



通过这些步骤，Keras 自动处理反向传播的过程，更新模型的权重，以最小化损失函数。